In [407]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

Defs:

In [464]:
# def for checking str in every file
def find_str_in_all_files(str_to_find):
    in_dict = {}
    for count in range(len(files_df)):
        str_to_find_lower = str_to_find.strip().lower()
        file = files_df.loc[count,'files']
        data = pd.read_excel(f'{files_df.loc[count,"folder"]}\\{file}', sheet_name=sheet_name)
        try:
            str_index = data[data.apply(lambda row: row.astype(str).str.lower().str.contains(str_to_find_lower).any(), axis=1)].index.item()
            in_dict[file] = str_index
        except:
            continue
    return files_df.shape[0], len(in_dict.keys()), in_dict

# def for finfing str index
def find_id_str(str_to_find):
    str_to_find_lower = str_to_find.strip().lower()
    id_str = new_data[new_data.apply(lambda row: row.astype(str).str.lower().str.contains(str_to_find_lower).any(), axis=1)].index.item()
    return id_str

def diff_by_cols(data_col_names, data_merge, count_start_col=1):
    
    # cols lisr without name col (first)
    cols_list = data_col_names.columns.tolist()[count_start_col:]

    for col in cols_list:
        # choose only cols with previous col in name
        col_cols_list = [x for x,y in zip(data_merge.columns,data_merge.columns.str.contains(col)) if y==True]
        assert len(col_cols_list) == 2
        # print diff
        data_merge[f'{col}_diff'] = -1*(data_merge[col_cols_list[0]] - data_merge[col_cols_list[1]])
        # print pct diff
        data_merge[f'{col}_pct_diff'] = 0 
        # condition = abs(data_merge[[col_cols_list[0],col_cols_list[1]]]).max(axis=1)!=0
        # data_merge.loc[condition, f'{col}_pct_diff'] = data_merge.loc[condition,f'{col}_diff'] / abs(data_merge.loc[condition,[col_cols_list[0],col_cols_list[1]]]).max(axis=1)
        condition = data_merge[col_cols_list[0]]!=0
        data_merge.loc[condition, f'{col}_pct_diff'] = abs(data_merge.loc[condition,f'{col}_diff'] / data_merge.loc[condition,col_cols_list[0]])
        data_merge[f'{col}_pct_diff'] = np.sign(data_merge[f'{col}_diff']) * data_merge[f'{col}_pct_diff']

    return data_merge

def del_null_data(data):
    list_to_del = [np.nan, '0', 0]
    # del empty rows
    data = data[(~data.isin(list_to_del)).any(axis=1)]
    # del empty cols
    cols_list = (~data.isin(list_to_del)).any(axis=0)
    cols_list = cols_list[cols_list==True].index.tolist()
    data = data[cols_list]
    return data

def data_to_parts(data,need_to_name=False):
    string_to_find = 'TOTAL'
    string_to_find = string_to_find.strip().lower()
    try:
        assert data.iloc[0,5].strip().lower() == string_to_find
        part_col = 6
    except:
        part_col = 5
    asserts_part = data.iloc[:,:part_col]
    asserts_part.columns = ['name'] + asserts_part.iloc[0,:].tolist()[1:]
    if need_to_name == True:
        asserts_part['class_name_1'] = 'Assets'
        asserts_part = asserts_part.iloc[2:,:]
    else:
        asserts_part = asserts_part.iloc[1:,:]

    liabilities_part = data.iloc[:,part_col:part_col+part_col]
    liabilities_part.columns = ['name'] + liabilities_part.iloc[0,:].tolist()[1:]
    if need_to_name == True:
        liabilities_part['class_name_1'] = 'Liabilities and equity'
        liabilities_part = liabilities_part.iloc[2:,:]
    else:
        liabilities_part = liabilities_part.iloc[1:,:]

    data = pd.concat([asserts_part,liabilities_part],axis=0).reset_index(drop=True)
    return data

Variables

In [409]:
print_to_excel = True

sheet_name = 'FX Exposure_Balance Sheet'
month = ['nov','dec']
month = month[0]

In [410]:
folders = [x for x in os.listdir() if month in x]
files_list = []
folders_for_df = []
for count in range(len(folders)):
    files_folder_list = os.listdir(f'{folders[count]}')
    files_folder_list = [x for x in files_folder_list if '~' not in x]
    files_list += files_folder_list
    folders_for_df += [folders[count] for x in range(len(files_folder_list))]
files_list

['BRFTO FX Exposure Model 28.11.2024 - With Pnl & Tax Credit -New Model.xlsx',
 'BRSFT FX Exposure Model 28.11.2024 - With Pnl & Tax Credit.xlsx',
 'Cópia de BRFHRG FX Exposure 28.11.2024 - With pnl  Tax Credit - New model.xlsx',
 'BRFHRG FX Exposure 29.11.2024 - With pnl  Tax Credit - New model.xlsx',
 'BRFTO FX Exposure Model 29.11.2024 - With Pnl & Tax Credit -New Model.xlsx',
 'BRSFT FX Exposure Model 29.11.2024 - With Pnl & Tax Credit.xlsx']

In [411]:
files_df = pd.DataFrame({'files':files_list, 'folder':folders_for_df})
files_df['day'] = files_df.files.str.extract(r'([0-9]{2}.[0-9]{2}.[0-9]{4})')
files_df['company'] = files_df.files.str.extract(r'(BR\w* )')
files_df

,files,folder,day,company
0,BRFTO FX Exposure Model 28.11.2024 - With Pnl ...,28 nov,28.11.2024,BRFTO
1,BRSFT FX Exposure Model 28.11.2024 - With Pnl ...,28 nov,28.11.2024,BRSFT
2,Cópia de BRFHRG FX Exposure 28.11.2024 - With ...,28 nov,28.11.2024,BRFHRG
3,BRFHRG FX Exposure 29.11.2024 - With pnl Tax ...,29 nov,29.11.2024,BRFHRG
4,BRFTO FX Exposure Model 29.11.2024 - With Pnl ...,29 nov,29.11.2024,BRFTO
5,BRSFT FX Exposure Model 29.11.2024 - With Pnl ...,29 nov,29.11.2024,BRSFT


In [412]:
files_df_merge = files_df.merge(files_df, on='company', suffixes=['_old','_new'])
files_df_couple = files_df_merge[(files_df_merge.day_old!=files_df_merge.day_new)&(files_df_merge.day_old<files_df_merge.day_new)]
files_df_couple = files_df_couple.reset_index(drop=True)
files_df_couple

,files_old,folder_old,day_old,company,files_new,folder_new,day_new
0,BRFTO FX Exposure Model 28.11.2024 - With Pnl ...,28 nov,28.11.2024,BRFTO,BRFTO FX Exposure Model 29.11.2024 - With Pnl ...,29 nov,29.11.2024
1,BRSFT FX Exposure Model 28.11.2024 - With Pnl ...,28 nov,28.11.2024,BRSFT,BRSFT FX Exposure Model 29.11.2024 - With Pnl ...,29 nov,29.11.2024
2,Cópia de BRFHRG FX Exposure 28.11.2024 - With ...,28 nov,28.11.2024,BRFHRG,BRFHRG FX Exposure 29.11.2024 - With pnl Tax ...,29 nov,29.11.2024


Counting

In [465]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_Brazil_monitoring.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl', mode='w') as writer:
        files_df_couple.to_excel(writer, index=False, sheet_name='all_files')

for string_id in files_df_couple.index:
    # try:
    # read old data
    old_path = f'{files_df_couple.loc[string_id, "folder_old"]}\\{files_df_couple.loc[string_id, "files_old"]}'
    old_data = pd.read_excel(old_path, sheet_name=sheet_name)
    old_data_date = files_df_couple.loc[string_id, "day_old"]
    print(old_path)

    # read new data
    new_path = f'{files_df_couple.loc[string_id, "folder_new"]}\\{files_df_couple.loc[string_id, "files_new"]}'
    new_data = pd.read_excel(new_path, sheet_name=sheet_name)
    new_data_date = files_df_couple.loc[string_id, "day_new"]
    print(new_path)

    company_name = files_df_couple.loc[string_id, "company"]

    count_data_date = 0
    for data_date in [old_data, new_data]:
        
        count_data_date += 1



        ### DATA_1
        # 1)	сначала видеть изменение общей позиции, суммарной день ко 
        # 2)	затем видеть изменение on-Balance и Off-Balance
        start_str = 'USD Equivalent'
        end_str = 'Net Exposure Balance BRL'

        data_1 = data_date.loc[find_id_str(start_str):find_id_str(end_str),]
        data_1 = del_null_data(data_1)
        data_1 = data_1.iloc[:,:3].reset_index(drop=True)
        data_1.columns = ['name'] + data_1.iloc[0,1:].values.tolist()

        ### DATA_3
        # 3)	затем видеть изменение итоговых балансовых статей:
        start_str = 'USD denominated'
        end_str = 'Total assets'
        data_3 = data_date.loc[[find_id_str(start_str),find_id_str(end_str)],]
        data_3 = del_null_data(data_3)

        data_3 = data_to_parts(data_3)

        ### DATA_4
        # 4)	затем видеть изменения внебалансовых статей (их немного)
        start_str = 'Total assets'
        end_str = 'Total Off Balance Assets'
        pre_data_4 = data_date.loc[find_id_str(start_str):find_id_str(end_str),]

        start_str = 'Assets'
        header_str = 'USD denominated'
        start_id = pre_data_4.loc[pre_data_4['Unnamed: 1']=='Assets',:].index.item()
        data_4 = pre_data_4.loc[start_id:]

        header_df = data_date.loc[find_id_str(header_str):find_id_str(header_str)+1,]
        data_4 = pd.concat([header_df,data_4],axis=0)
        data_4 = del_null_data(data_4)

        data_4 = data_to_parts(data_4, True)

        data_4 = data_4[~data_4.iloc[:,0].isna()].reset_index(drop=True)
        # del empty cols
        cols_list = (~data_4.isin(list_to_del)).any(axis=0)
        cols_list = cols_list[cols_list==True].index.tolist()
        data_4 = data_4[cols_list]

        data_4 = data_4[['class_name_1'] + data_4.columns.tolist()[:-1]]

        ### DATA_5
        # 5)	затем видеть детальное изменение балансовых статей.
        start_str = 'USD denominated'
        end_str = 'Total assets'
        data_5 = data_date.loc[find_id_str(start_str):find_id_str(end_str)-1,]
        data_5 = del_null_data(data_5)

        data_5 = data_to_parts(data_5, True)

        data_5 = data_5[~data_5.iloc[:,0].isna()].reset_index(drop=True)

        class_name_list = ['Current assets','Non-current assets','Current liabilities','Non-current liabilities','Equity']
        class_name_list_lower = list(map(lambda x: x.strip().lower(), class_name_list))
        data_5['class_name_2'] = np.nan
        condition = data_5.name.str.strip().str.lower().isin(class_name_list_lower)
        data_5.loc[condition, 'class_name_2'] = data_5.loc[condition, 'name']
        data_5['class_name_2'] = data_5['class_name_2'].fillna(method='ffill')
        data_5 = data_5[(data_5.name!=data_5.class_name_2)|(data_5.name=='Equity')].reset_index(drop=True)

        data_5 = data_5[['class_name_1', 'class_name_2'] + data_5.columns.tolist()[:-2]]
        

        if count_data_date == 1:
            data_1_old = data_1
            data_3_old = data_3
            data_4_old = data_4
            data_5_old = data_5
        if count_data_date == 2:
            data_1_new = data_1
            data_3_new = data_3
            data_4_new = data_4
            data_5_new = data_5

    # concat old and new data
    ### data_1
    data_1_merge = data_1_old.merge(data_1_new, how='outer', on='name', suffixes=[f'_{old_data_date}',f'_{new_data_date}'])
    data_1_merge = data_1_merge[~data_1_merge.name.isna()]
    data_1_merge = diff_by_cols(data_1_new,data_1_merge)

    ### data_3
    data_3_merge = data_3_old.merge(data_3_new, how='outer', on='name', suffixes=[f'_{old_data_date}',f'_{new_data_date}'])
    data_3_merge = diff_by_cols(data_3_new,data_3_merge)

    ### data_4
    data_4_merge = data_4_old.merge(data_4_new, how='outer', on=['class_name_1','name'], suffixes=[f'_{old_data_date}',f'_{new_data_date}'])
    data_4_merge = data_4_merge.fillna(0)
    data_4_merge = diff_by_cols(data_4_new,data_4_merge,2)

    ### data_5
    data_5_merge = data_5_old.merge(data_5_new, how='outer', on=['class_name_1', 'class_name_2','name'], suffixes=[f'_{old_data_date}',f'_{new_data_date}'])
    data_5_merge = data_5_merge.fillna(0)
    data_5_merge = diff_by_cols(data_5_new,data_5_merge,3)

    ### TO EXCEL
    if print_to_excel == True:
        with pd.ExcelWriter(Output_file, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            count_row = 0
            pd.DataFrame({'company':company_name},index=[0]).to_excel(writer, header=False, index=False, sheet_name=company_name, startrow=count_row)
            count_row += 2
            files_df_couple.loc[string_id:string_id,:].to_excel(writer, header=True, index=False, sheet_name=company_name, startrow=count_row)
            count_row += 3
            data_1_merge.to_excel(writer, header=True, index=False, sheet_name=company_name, startrow=count_row)
            count_row += len(data_1_merge) + 2
            data_3_merge.to_excel(writer, header=True, index=False, sheet_name=company_name, startrow=count_row)
            count_row += len(data_3_merge) + 2
            data_4_merge.to_excel(writer, header=True, index=False, sheet_name=company_name, startrow=count_row)
            count_row += len(data_4_merge) + 2
            data_5_merge.to_excel(writer, header=True, index=False, sheet_name=company_name, startrow=count_row)




    # except Exception as e:
    #     print(e, 'in string ',string_id)

28 nov\BRFTO FX Exposure Model 28.11.2024 - With Pnl & Tax Credit -New Model.xlsx
29 nov\BRFTO FX Exposure Model 29.11.2024 - With Pnl & Tax Credit -New Model.xlsx


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10068\113402984.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asserts_part['class_name_1'] = 'Assets'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10068\113402984.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liabilities_part['class_name_1'] = 'Liabilities and equity'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10068\643082407.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a fu

28 nov\BRSFT FX Exposure Model 28.11.2024 - With Pnl & Tax Credit.xlsx
29 nov\BRSFT FX Exposure Model 29.11.2024 - With Pnl & Tax Credit.xlsx


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10068\113402984.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asserts_part['class_name_1'] = 'Assets'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10068\113402984.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liabilities_part['class_name_1'] = 'Liabilities and equity'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10068\643082407.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a fu

28 nov\Cópia de BRFHRG FX Exposure 28.11.2024 - With pnl  Tax Credit - New model.xlsx


Exception ignored in: <function ZipFile.__del__ at 0x0000014DB04D8CC0>
Traceback (most recent call last):
  File "c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\zipfile.py", line 1874, in __del__
    self.close()
  File "c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\zipfile.py", line 1891, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


29 nov\BRFHRG FX Exposure 29.11.2024 - With pnl  Tax Credit - New model.xlsx


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10068\113402984.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asserts_part['class_name_1'] = 'Assets'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10068\113402984.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liabilities_part['class_name_1'] = 'Liabilities and equity'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10068\643082407.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a fu

check